In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_monthly (
calmonth DATE,
store_id STRING,
billcount BIGINT,
billcount_mdl BIGINT,
billcount_fresh BIGINT,
billcount_mbs BIGINT,
tpay_trans FLOAT,
rev_amt_pos_mbs FLOAT,
rev_amt_pos_mld_mbs FLOAT
)
USING delta
TBLPROPERTIES (
  'delta.enableDeletionVectors' = 'true',
  'delta.feature.deletionVectors' = 'supported')""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_monthly 
WHERE calmonth >= add_months(DATE_TRUNC('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), -1)
""")

In [0]:
spark.sql(f"""INSERT INTO {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_billcount_store_monthly
WITH header AS (
    select DISTINCT LTRIM(transaction_no, '0') transaction_no 
    from {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_loyalty a 
    WHERE calday >= add_months(DATE_TRUNC('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), -1)
),
f_sale_prod AS (
    SELECT
        DATE_TRUNC('MONTH', calday) calmonth,
        store_id,
        product_id,
        transaction_type,
        LTRIM(transaction_no, '0') transaction_no,
        SUM(sale_amount - tax_amount + IF(discount_type IN ('ZRTA', 'ZCRT'), discount_value, 0)) rev_amt_pos,
        SUM(CASE WHEN payment_method IN ('TQRP', 'ZTPA') THEN tender_value ELSE 0 END) tpay_trans
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod
    WHERE calday >= add_months(DATE_TRUNC('MONTH', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), -1)
    GROUP BY 1, 2, 3, 4, 5
),
tpay AS (
  SELECT
    a.calmonth,
    a.store_id,
    SUM(tpay_trans) tpay_trans
FROM f_sale_prod a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
WHERE c.business_unit ='1500'
    AND c.store_id not like '195%'
GROUP BY 1, 2
),
main AS (
SELECT
    a.calmonth, a.store_id,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 THEN a.transaction_no END) billcount,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND product.manufacturer_id in ('2325', '1915') THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND product.manufacturer_id in ('2325', '1915') THEN a.transaction_no END) billcount_mdl,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND product.mch2_id = '101' THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND product.mch2_id = '101' THEN a.transaction_no END) billcount_fresh,
    COUNT(DISTINCT CASE WHEN b.bill_count = 1 AND header.transaction_no IS NOT NULL  THEN a.transaction_no END) - COUNT(DISTINCT CASE WHEN b.bill_count = -1 AND header.transaction_no IS NOT NULL THEN a.transaction_no END) billcount_mbs,
    SUM(CASE WHEN header.transaction_no IS NOT NULL THEN a.rev_amt_pos ELSE 0 END) rev_amt_pos_mbs,
    SUM(CASE WHEN header.transaction_no IS NOT NULL AND product.manufacturer_id IN ('2325', '1915') THEN a.rev_amt_pos ELSE 0 END) rev_amt_pos_mld_mbs
FROM f_sale_prod a
INNER JOIN {catalog_name}.udp_wcm_silver_dim.transaction_type b ON a.transaction_type = b.transaction_type
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product ON a.product_id = product.product_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
LEFT JOIN header ON LTRIM(a.transaction_no, '0') = header.transaction_no
WHERE c.business_unit ='1500'
    AND c.store_id not like '195%'
    AND product.mch2_id IN ('101', '102', '201', '202', '203')
GROUP BY 1, 2
)
SELECT IFNULL(main.calmonth, tpay.calmonth) calmonth,
    IFNULL(main.store_id, tpay.store_id) store_id,
    main.billcount,
    main.billcount_mdl,
    main.billcount_fresh,
    main.billcount_mbs,
    tpay.tpay_trans,
    main.rev_amt_pos_mbs,
    main.rev_amt_pos_mld_mbs
FROM main
FULL JOIN  tpay USING (calmonth, store_id)
;
""")